In [3]:
# git clone - prepare training
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r /kaggle/working/yolov5/requirements.txt

  Obtaining dependency information for ultralytics>=8.0.232 from https://files.pythonhosted.org/packages/41/8f/37e7c14912a504df76212ff93eb8b78047fa2b3318dece8c2da6192231be/ultralytics-8.0.235-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 5.8 MB/s eta 0:00:0000:0100:01


In [2]:
# Load trained model - 10 epochs
!pip install wandb
import wandb

wandb.login(key = "c8767797aae76cbcd389ff29929ace1ac3021161")    # key's DoanNgocCuong
run = wandb.init()
artifact = run.use_artifact('doanngoccuong_nh/FaceDetection_Yolov5/facedet_widerface_cfgyolov5_colab:v0', type='model')
artifact_dir = artifact.download()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [ ]:
# Write again file custom_dataset.yaml (because file trong dataset is for kaggle). 

# Nội dung của file .yaml
yaml_content = """

# train: /content/drive/MyDrive/colab/custom_dataset/images_and_labels/images/train
# val: /content/drive/MyDrive/colab/custom_dataset/images_and_labels/images/val
train: /kaggle/input/facedetection-widerfacedataset-yolov5-zip/custom_dataset/images_and_labels/images/train
val: /kaggle/input/facedetection-widerfacedataset-yolov5-zip/custom_dataset/images_and_labels/images/val

# number of classes
nc: 1
# class names
names: ['face']
"""
# Tạo và ghi (w ghi mới, a: append ghi thêm)
with open('/kaggle/working/custom_dataset.yaml', 'w') as file: 
    file.write(yaml_content)
print("custom_dataset.yaml file created successfully!")

In [ ]:
# --resume flag is for continuing training that was interrupted,  should not change any settings from your initial training command, such as --epochs, --batch, or --data.
# --weights flag is for starting a new training based on a pre-trained model.

# Kaggle GPU T4x2 mạnh gấp đôi GPU thì batch *2 lên from 16->32
# cache for load trainloader, testloader. name for name run in wandb
# SUCCESSFUL
%cd /kaggle/working/yolov5
!python train.py --img 640 --batch 32 --epochs 100 \
  --data /kaggle/working/custom_dataset.yaml \
  --weights '/kaggle/working/artifacts/facedet_widerface_cfgyolov5_colab:v0/best.pt' \
  --project FaceDetection_Yolov5   --name facedet_widerds_yolov5s_kaggle_ep10to100 \
  --entity doanngoccuong_nh \
  --cache 
wandb.finish() # Kết thúc quá trình Train model - TẮT CHẤM XANH WANDB

# https://wandb.ai/doanngoccuong_nh/FaceDetection_Yolov5/artifacts/model/facedet_widerface_cfgyolov5_colab/v0
# --weights 'wandb-artifact://<entity>/<project>/model:v0' --name custom_model --cache \
# UN SUCCESSFUL
# CODE KO HOẠT ĐỘNG KHI LOAD TRỰC TIẾP TỪ WANDB
# !python train.py --img 640 --batch 32 --epochs 100 \
#   --data /kaggle/working/custom_dataset.yaml \
#   --weights 'wandb-artifact://doanngoccuong_nh/FaceDetection_Yolov5/facedet_widerface_cfgyolov5_colab:v0' \
#   --project FaceDetection_Yolov5   --name facedet_widerds_yolov5s_kaggle_ep10to100 \
#   --entity doanngoccuong_nh \
#   --cache 


Đồ thị wandb cho thấy, sau 20 epochs, model cải thiện nhưng ko đáng kể so với tốc độ cải thiện của 10 epochs đầu (Cũng cho thấy 10 epochs đầu quá ukii)